#### Ensemble learning với thuật toán LVQ cơ bản sử dụng tỉ lệ mẫu trong từng neuron

In [1]:
import numpy as np
from sklearn.metrics import euclidean_distances
import pandas as pd

import sys
sys.path.insert(0, '../SOM-LVQ')
import LVQ
import weighted_neurons

In [2]:
data = pd.read_csv(r"../data/UCI/bupa.data.csv", header=None)

In [3]:
threshold = data[5].values

label = threshold >= 3

label = label * 1

In [4]:
x = data.iloc[:, 0:5].values
y = label
from sklearn.cross_validation import train_test_split
x_train = x[data[6] == 2]
y_train = label[data[6] == 2]
x_test = x[data[6] == 1]
y_test = label[data[6] == 1]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# use Minmaxscaler because we use euclidean distance
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x_train = minmax.fit_transform(x_train)
x_test = minmax.transform(x_test)

C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
lvq1 = LVQ.LVQ(x=x_train, y=y_train, n_classes=2, n_neurons=50, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)

neurons = lvq1.train_LVQ2(x_train, y_train)

In [6]:
predicted_y = [lvq1.predict(instance) for instance in x_test]

from sklearn.metrics import classification_report

print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.59      0.61      0.60        79
          1       0.52      0.50      0.51        66

avg / total       0.56      0.56      0.56       145



C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))


In [7]:
lvq2 = LVQ.LVQ(x=x_train, y=y_train, n_classes=2, n_neurons=50, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)

neurons = lvq2.train_LVQ2(x_train, y_train)

In [8]:
predicted_y = [lvq2.predict(instance) for instance in x_test]

from sklearn.metrics import classification_report

print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.60      0.51      0.55        79
          1       0.50      0.59      0.54        66

avg / total       0.55      0.54      0.55       145



C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))


In [9]:
lvq3 = LVQ.LVQ(x=x_train, y=y_train, n_classes=2, n_neurons=50, epsilon=0.9, p_vectors=[], epsilon_dec_factor=0.001)

neurons = lvq3.train_LVQ2(x_train, y_train)

In [10]:
predicted_y = [lvq3.predict(instance) for instance in x_test]

from sklearn.metrics import classification_report

print (classification_report(y_test, predicted_y, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.55      0.53      0.54        79
          1       0.46      0.47      0.46        66

avg / total       0.50      0.50      0.50       145



C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))


#### Thuật toán LVQ được thêm vào các hàm tính win_map và tỉ lệ các mẫu trong từng neuron

In [11]:
mapping1 = lvq1.win_map_LVQ(x_train)
mapping2 = lvq2.win_map_LVQ(x_train)
mapping3 = lvq3.win_map_LVQ(x_train)

In [12]:
propa1 = lvq1.propabilityLVQ()
propa2 = lvq2.propabilityLVQ()
propa3 = lvq3.propabilityLVQ()

In [20]:
result = np.array([])

for sample in x_test:
    pos1 = lvq1.find_closest(sample)
    pos2 = lvq2.find_closest(sample)
    pos3 = lvq3.find_closest(sample)
    l1 = len(mapping1[pos1[0]])
    l2 = len(mapping2[pos2[0]])
    l3 = len(mapping3[pos3[0]])
    pro1 = (l1*propa1[0, pos1[0]] + l2*propa2[0, pos2[0]] + l3*propa3[0, pos3[0]])/(l1+l2+l3)
    pro2 = (l1*propa1[1, pos1[0]] + l2*propa2[1, pos2[0]] + l3*propa3[1, pos3[0]])/(l1+l2+l3)
    tmp = np.argmax(np.array([pro1, pro2]))
    result = np.append(result, tmp)

C:\Users\Thend\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Thend\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [19]:
# result = []
# for sample in x_test:
#     t = weighted_neurons.weighted_neurons_predict(models=[lvq1, lvq2, lvq3], 
#                                                   mappings=[mapping1, mapping2, mapping3], 
#                                                   propa=[propa1, propa2, propa3], 
#                                                   y = sample)
#     tmp_result.append(t)

In [21]:
print (classification_report(y_test, result, target_names=['0', '1', '2', '3', '4', '5', '6']))

             precision    recall  f1-score   support

          0       0.62      0.63      0.63        79
          1       0.55      0.53      0.54        66

avg / total       0.59      0.59      0.59       145



C:\Users\Thend\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
